In [ ]:
import os
from PIL import Image
import numpy as np

openslide_path=r'C:\Users\Wirtz\Documents\openslide-win64-20221111\bin'
# os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
from openslide import OpenSlide

In [ ]:
wsisrc = r'\\fatherserverdw\ashleyex\Pancreas images\Fatemeh PDAC\Katsuya S2\PDAC-VI-KH-002'
tenxsrc = os.path.join(wsisrc, '10x_python')
twosrc = os.path.join(wsisrc, '1x_python')
if not os.path.exists(tenxsrc):
    os.mkdir(tenxsrc)
if not os.path.exists(twosrc):
    os.mkdir(twosrc)

In [ ]:
wsis = [_ for _ in os.listdir(wsisrc) if _.endswith('ndpi')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]
tenx = [_ for _ in os.listdir(tenxsrc) if _.endswith('tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]
missingimages = [_ for _ in wsis2 if _ not in tenx2]

In [ ]:
for idx,missingimage in enumerate(missingimages):
    print(idx,'/',len(missingimages),' processing: ',missingimage)
    wsi = OpenSlide(os.path.join(wsisrc,missingimage+'.ndpi'))
    rsf = 1
    svs_img = wsi.read_region(location=(0,0),level=0,size=wsi.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf/float(wsi.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(tenxsrc,missingimage+'.tif'),resolution=1,resolution_unit=1,quality=100,compression=None)

    rsf = 4
    resize_factorx = 1/rsf #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = 1/rsf
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)

0 / 108  processing:  PDAC-VI-KH-002_0103


ValueError: image has wrong mode

In [ ]:
    rsf = 8
    resize_factorx = rsf/float(wsi.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img2_5x = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img2_5x.save(os.path.join(twosrc,missingimage+'.tif'),resolution=2_5,resolution_unit=1,quality=100,compression=None)